In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp

In [ ]:
from datafold.appfold import EDMD
from datafold.dynfold import DMDFull
from datafold.dynfold.transform import TSCRadialBasis
from datafold.pcfold import InverseMultiquadricKernel, PCManifold, TSCDataFrame

## Inverted pendulum Physics

In [ ]:
from datafold.utils.kmpc import InvertedPendulum

## Data generation

In [ ]:
# Data generation parameters
sim_time_step = 0.01  # s
sim_num_steps = 1000  # -
training_size = 20  # -

Different options exist about stacking the trajectories. We start with stacking them into one very long trajectory. (Also to consider reusing the final states, currently resetting to 0).

In [ ]:
invertedPendulum = InvertedPendulum()

X = np.zeros((4, training_size * sim_num_steps))
Y = np.zeros((4, training_size * sim_num_steps))
U = np.zeros((1, training_size * sim_num_steps))
Xlist, Ulist = [], []
xycols = ["x", "xdot", "theta", "thetadot"]
np.random.seed(42)
for i in range(training_size):
    control_amplitude = 0.1 + 0.9 * np.random.random()
    control_frequency = np.pi + 2 * np.pi * np.random.random()
    control_phase = 2 * np.pi * np.random.random()
    control_func = lambda t, y: control_amplitude * np.sin(
        control_frequency * t + control_phase
    )
    invertedPendulum.reset()
    traj = invertedPendulum.trajectory(sim_time_step, sim_num_steps, control_func)
    assert (
        invertedPendulum.sol.success
    ), f"Divergent solution for amplitude={control_amplitude}, frequency={control_frequency}"
    t = invertedPendulum.sol.t
    dfx = pd.DataFrame(data=traj.T, index=t, columns=xycols)
    dfx["u"] = 0.0
    Xlist.append(dfx)
    control_input = control_func(t, traj)
    dfu = pd.DataFrame(data=control_input, index=t, columns=("u",))
    for col in xycols:
        dfu[col] = 0.0
    dfu = dfu[xycols + ["u"]]
    Ulist.append(dfu)

    X[:, i * sim_num_steps : (i + 1) * sim_num_steps] = traj[:, :-1]
    Y[:, i * sim_num_steps : (i + 1) * sim_num_steps] = traj[:, 1:]
    U[:, i * sim_num_steps : (i + 1) * sim_num_steps] = control_func(
        invertedPendulum.sol.t[:-1], traj[:, :-1]
    )

XU = TSCDataFrame.from_frame_list(Xlist + Ulist)

In [ ]:
XU

In [ ]:
plt.figure(figsize=(16, 3))
plt.plot(X[0].ravel())

In [ ]:
plt.figure(figsize=(16, 3))
plt.plot(U[0].ravel())

In [ ]:
X_tsc = TSCDataFrame.from_frame_list(Xlist)[["x", "xdot", "theta", "thetadot"]]
X_tsc["u"] = TSCDataFrame.from_frame_list(Ulist)[["u"]]
X_tsc

## DMD only

In [ ]:
dmd = DMDFull()
dmd.fit(X_tsc, store_system_matrix=True)
plt.imshow(dmd.koopman_matrix_)

## EDMD Predictor

In [ ]:
num_rbfs = 100

In [ ]:
from datafold.utils.kmpc import InverseQuadraticKernel

In [ ]:
EDMD()